# Google Earth Engine - Machine Learning Tutorial

### This notebook is a Tutorial of how to use machine learning models in Google Earth Engine

### Table of Contents
* [I. Variables](#I.-Variables)
* [II. Dataset](#II.-Dataset)
* [III. Training](#III.-Training)
* [IV. Evaluation](#IV.-Evaluation)
* [V. Inference](#V.-Inference)


In [ ]:
#RUN ONLY IF IN GOOGLE COLAB ENVIRONMENT
from google.colab import drive
drive.mount('/content/gdrive')
%cd gdrive/My Drive/Colab Notebooks/Leakmited/france

Mounted at /content/gdrive
/content/gdrive/My Drive/Colab Notebooks/Leakmited/france


In [ ]:
# Import, authenticate and initialize the Earth Engine library.
import ee
ee.Authenticate()
ee.Initialize()

# I. Variables

In [ ]:
start_date = "2020-01-01"
end_date   = "2020-12-31"
landsat    = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B10', 'B11']
sentinel   = ['VV','VH','VV_1','VH_1']
bands      = landsat+sentinel
scale      = 30
nb_trees   = 50

# II. Dataset

First you will have to construct the dataset by using Google Earth Engine Code Editor.  
Using the imagery as guidance, hover over the `Geometry Imports` box next to the geometry drawing tools and click `+ new layer`. Each new layer represents one class within the training data.  
Let the first new layer represent `urban`. 
* Locate points in the new layer in urban or built up areas (buildings, roads, parking lots, etc.). 
* When finished collecting points, click `Exit` and configure the import (top of the script) as follows : 
  * Name the layer `urban` and click the icon to configure it. 
  * `Import as` FeatureCollection. 
  * `Add property` landcover and set its value to 2. (Subsequent classes will be 0 for field, 1 for forest, 3 for water) 
  * When finished, click `OK`  
<img src="tuto_images/18.PNG" width=500>
* Export the Feature Collection to Assets by running the following on the Code Editor:
  ```
  Export.table.toAssets({
    collection:urban,
    description:'urban_pixels',
    assetId:'urban})
  ``` 
* Do the same for the other classes





In [ ]:
def get_landsat(start_date,end_date):
  def addQualityBands(image):
    return image.addBands(image.metadata('system:time_start'))
  collection = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')
  collection = collection.filterDate(start_date,end_date).filterMetadata('CLOUD_COVER','less_than',0.3).map(addQualityBands)
  image = collection.qualityMosaic('system:time_start').setDefaultProjection(collection.first().select(bands[0]).projection())
  return image

def get_sentinel1(start_date,end_date):
  collection = ee.ImageCollection("COPERNICUS/S1_GRD")
  collection = collection.filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')).filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
  collection = collection.select('VV','VH').filterDate(start_date,end_date)
  asc  = collection.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING')).median()
  desc = collection.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING')).median()
  image = ee.Image.cat([asc,desc])
  image = image.clamp(-50,1).subtract(-50).divide(51).setDefaultProjection(collection.first().projection())
  return image

def get_image(start_date,end_date,landsat_bands,sentinel1_bands):
  landsat = get_landsat(start_date,end_date)
  sentinel = get_sentinel1(start_date,end_date)
  image = ee.Image.cat([landsat.select(landsat_bands),sentinel.select(sentinel1_bands)])
  return image

def get_labels():  
  field = ee.FeatureCollection('users/leakm/field')
  forest = ee.FeatureCollection('users/leakm/forest')
  urbain = ee.FeatureCollection('users/leakm/urbain')
  water = ee.FeatureCollection('users/leakm/water')
  labels = field.merge(forest).merge(urbain).merge(water)
  return labels
  
def get_training_data(image,labels,scale):
  #Sample the input imagery to get a FeatureCollection of training data.
  training = image.sampleRegions(
    collection = labels,
    properties = ['landcover'],
    tileScale  = 8,
    scale      = scale)
  return training

In [ ]:
image = get_image(start_date,end_date,landsat,sentinel)
labels = get_labels()
training = get_training_data(image,labels,scale)

# III. Training

In [ ]:
def train(training,bands,nb_trees):
  rf = ee.Classifier.smileRandomForest(numberOfTrees=nb_trees)
  classifier = rf.train(
    features        = training,
    classProperty   = 'landcover',
    inputProperties = bands
  )
  confusion_matrix = classifier.confusionMatrix()
  print('RF error matrix: ', confusion_matrix.getInfo())
  print('RF accuracy: ', confusion_matrix.accuracy().getInfo())
  return classifier

In [ ]:
classifier = train(training,bands,nb_trees)

# IV. Evaluation

In [ ]:
def evaluate_model(image,classifier,nlcd,scale):
  evaluation = image.addBands(nlcd).sample(region=nlcd.geometry().bounds(),numPixels = 1000,seed = 0,scale=scale)
  evaluated = evaluation.classify(classifier)
  testAccuracy = evaluated.errorMatrix('true_landcover', 'classification')
  print('Validation error matrix: ', testAccuracy.getInfo())
  print('Validation overall accuracy: ', testAccuracy.accuracy().getInfo())

In [ ]:
# Use a known dataset of labels if you don't want to construct the evaluation dataset by hand
nlcd = ee.Image('users/leakm/france2017').select('b1').remap([42,41,43,44,31,32,221,222,11,12,34,36,211,45,46,51,53],[2,2,2,2,1,1,0,0,0,0,0,0,0,3,0,3,3]).rename('landcover')
evaluate_model(image,classifier,nlcd,scale)

# V. Inference

In [ ]:
def infere(image,region,classifier):
  #Classify the input imagery.
  classified = image.clip(region.bounds()).classify(classifier)
  return classified,pipeline

def display(Map,classified,pipeline):
  #Define a palette for the Land Use classification.
  palette = [
    '008000', #  field (0) // green
    'D3D3D3', # forest (1)  // grey
    'FF0000', # urbain (2)  // red
    '0000FF'  # water (3)  // blue
  ]
  Map.addLayer(classified, {'min': 0, 'max': 2, 'palette': palette}, 'Land Use Classification')

def export_inference(classified,name,pipeline,scale):
  task = ee.batch.Export.image.toDrive(
    image          = classified,
    description    = name,
    region         = pipeline.geometry().bounds(),
    scale          = scale,
    fileNamePrefix = name
  )
  task.start()

In [ ]:
# Define your region here by writing ee.Geometry... 
# for example region = ee.Geometry.Rectangle([xmin,ymin,xmax,ymax]) with geospatial coordinates
# or region = ee.Geometry.Point([lon,lat]).buffer(radius) with radius in meters
region = ee.Geometry.Point([lon,lat]).buffer(radius)
# Inference
classified,pipeline = infere(image,region,classifier)

#Export inference as TIF to Drive
export_inference(classified,name+'water',pipeline,scale)

#Visualise inference on the Map
import geemap.efolium as gmap
Map = gmap.Map()
display_water(Map,classified,pipeline)
Map